In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 공통 전처리 셀

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

np.random.seed(0)
tf.random.set_seed(0)

# 1. 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Alcohol_Sales.csv')
data = df['S4248SM144NCEN'].values.reshape(-1, 1)

# 2. 정규화
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# 3. 시퀀스 만들기 (12개월 → 다음달)
window = 12
X, y = [], []
for i in range(len(data_scaled) - window):
    X.append(data_scaled[i:i+window])
    y.append(data_scaled[i+window])
X = np.array(X)
y = np.array(y)

# 4. train / test 분할
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

input_shape = (window, 1)


## 지난 주 실습 완성본

In [7]:
def build_lstm():
    model = models.Sequential([
        layers.LSTM(64, input_shape=input_shape),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_cnn():
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, padding='causal', activation='relu')(inp)
    x = layers.Conv1D(64, 3, padding='causal', activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    out = layers.Dense(1)(x)
    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

def build_cnn_lstm():
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, padding='causal', activation='relu')(inp)
    x = layers.Conv1D(64, 3, padding='causal', activation='relu')(x)
    x = layers.LSTM(64)(x)
    out = layers.Dense(1)(x)
    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

models_to_run = {
    "LSTM": build_lstm,
    "CNN": build_cnn,
    "CNN+LSTM": build_cnn_lstm,
}

results = {}
for name, builder in models_to_run.items():
    print(f"\n--- {name} ---")
    model = builder()
    model.fit(X_train, y_train,
              epochs=30,
              batch_size=8,
              validation_split=0.1,
              verbose=0)
    mse = model.evaluate(X_test, y_test, verbose=0)
    pred = model.predict(X_test)
    true_vals = scaler.inverse_transform(y_test)
    pred_vals = scaler.inverse_transform(pred)
    mae = mean_absolute_error(true_vals, pred_vals)
    results[name] = (mse, mae)
    print(f"{name} Test MSE: {mse:.6f} | MAE: {mae:.6f}")

print("\n[요약]")
for k, v in results.items():
    print(k, v)



--- LSTM ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 365ms/step
LSTM Test MSE: 0.025324 | MAE: 1671.799805

--- CNN ---
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
CNN Test MSE: 0.002250 | MAE: 480.322002

--- CNN+LSTM ---
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
CNN+LSTM Test MSE: 0.007042 | MAE: 817.839766

[요약]
LSTM (0.02532353810966015, 1671.7998046875)
CNN (0.0022495072335004807, 480.32200210813505)
CNN+LSTM (0.007041792385280132, 817.8397662450398)


## B안. 추가 실험 1 — CNN + SE 블록

In [8]:
def se_block(inputs, reduction=8):
    ch = inputs.shape[-1]
    x = layers.GlobalAveragePooling1D()(inputs)
    x = layers.Dense(ch // reduction, activation='relu')(x)
    x = layers.Dense(ch, activation='sigmoid')(x)
    x = layers.Reshape((1, ch))(x)
    return layers.Multiply()([inputs, x])

inp = layers.Input(shape=input_shape)
x = layers.Conv1D(64, 3, padding='causal', activation='relu')(inp)
x = se_block(x)   # SE 추가
x = layers.Conv1D(64, 3, padding='causal', activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
out = layers.Dense(1)(x)
se_model = models.Model(inp, out)
se_model.compile(optimizer='adam', loss='mse')

history = se_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=8,
    validation_split=0.1,
    verbose=0
)

mse = se_model.evaluate(X_test, y_test, verbose=0)
pred = se_model.predict(X_test)
true_vals = scaler.inverse_transform(y_test)
pred_vals = scaler.inverse_transform(pred)
mae = mean_absolute_error(true_vals, pred_vals)

print(f"SE-CNN Test MSE: {mse:.6f} | MAE: {mae:.6f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
SE-CNN Test MSE: 0.002672 | MAE: 545.476283


## C안. 추가 실험 2 — Transformer Encoder 기반 시계열

In [9]:
def transformer_encoder(inputs, num_heads=2, ff_dim=64, dropout=0.1):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    attn = layers.MultiHeadAttention(num_heads=num_heads,
                                     key_dim=inputs.shape[-1])(x, x)
    attn = layers.Dropout(dropout)(attn)
    x = layers.Add()([inputs, attn])

    y = layers.LayerNormalization(epsilon=1e-6)(x)
    y = layers.Dense(ff_dim, activation='relu')(y)
    y = layers.Dense(inputs.shape[-1])(y)
    y = layers.Dropout(dropout)(y)
    return layers.Add()([x, y])

inp = layers.Input(shape=input_shape)
x = transformer_encoder(inp, num_heads=2, ff_dim=64, dropout=0.1)
x = layers.GlobalAveragePooling1D()(x)
out = layers.Dense(1)(x)
trans_model = models.Model(inp, out)
trans_model.compile(optimizer='adam', loss='mse')

trans_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=8,
    validation_split=0.1,
    verbose=0
)

mse = trans_model.evaluate(X_test, y_test, verbose=0)
pred = trans_model.predict(X_test)
true_vals = scaler.inverse_transform(y_test)
pred_vals = scaler.inverse_transform(pred)
mae = mean_absolute_error(true_vals, pred_vals)

print(f"Transformer Encoder Test MSE: {mse:.6f} | MAE: {mae:.6f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Transformer Encoder Test MSE: 0.014161 | MAE: 1204.548270


## D안. 추가 실험 3 — CNN + CBAM

In [11]:
from tensorflow.keras import layers, models
import tensorflow as tf

# CBAM 1D
def cbam_block(inputs, reduction=8):
    ch = inputs.shape[-1]

    # ----- 1) Channel Attention -----
    avg_pool = layers.GlobalAveragePooling1D()(inputs)   # (batch, ch)
    max_pool = layers.GlobalMaxPooling1D()(inputs)       # (batch, ch)

    shared_dense = layers.Dense(ch // reduction, activation='relu')
    mlp_avg = shared_dense(avg_pool)
    mlp_max = shared_dense(max_pool)

    mlp_avg = layers.Dense(ch)(mlp_avg)
    mlp_max = layers.Dense(ch)(mlp_max)

    channel_attn = layers.Add()([mlp_avg, mlp_max])
    channel_attn = layers.Activation('sigmoid')(channel_attn)
    channel_attn = layers.Reshape((1, ch))(channel_attn)  # (batch, 1, ch)

    x = layers.Multiply()([inputs, channel_attn])  # (batch, time, ch)

    # ----- 2) Spatial Attention -----
    # 평균과 최대를 time축으로 모아서 concat -> conv1d
    avg_pool_spatial = layers.Lambda(lambda t: tf.reduce_mean(t, axis=-1, keepdims=True))(x)
    max_pool_spatial = layers.Lambda(lambda t: tf.reduce_max(t, axis=-1, keepdims=True))(x)
    spatial = layers.Concatenate(axis=-1)([avg_pool_spatial, max_pool_spatial])  # (batch, time, 2)
    spatial = layers.Conv1D(filters=1, kernel_size=3, padding='same', activation='sigmoid')(spatial)
    out = layers.Multiply()([x, spatial])
    return out

# ====== 모델 본체 ======
inp = layers.Input(shape=input_shape)
x = layers.Conv1D(64, 3, padding='causal', activation='relu')(inp)
x = cbam_block(x)          # 여기 이제 에러 안 남
x = layers.GlobalAveragePooling1D()(x)
out = layers.Dense(1)(x)
cbam_model = models.Model(inp, out)
cbam_model.compile(optimizer='adam', loss='mse')

cbam_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=8,
    validation_split=0.1,
    verbose=0
)

mse = cbam_model.evaluate(X_test, y_test, verbose=0)
pred = cbam_model.predict(X_test)
true_vals = scaler.inverse_transform(y_test)
pred_vals = scaler.inverse_transform(pred)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(true_vals, pred_vals)

print(f"CBAM-CNN Test MSE: {mse:.6f} | MAE: {mae:.6f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
CBAM-CNN Test MSE: 0.014771 | MAE: 1163.668697
